In [15]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.tokenizer import tokenize
from nltk.corpus import stopwords
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import SGDClassifier
from collections import Counter
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [2]:
# read the dataset
reviews_data = pd.read_json(r"C:\Users\user\Documents\UoM\Thesis\data\Clothing_Shoes_and_Jewelry_5.json", lines=True)

# Keep only the review text and the grade
reviews_data = reviews_data[['overall', 'reviewText']]


# Drop the products whose values are null
reviews_data = reviews_data[reviews_data['reviewText'].notna()]

In [3]:
ratings = []
for index,entry in enumerate(reviews_data['overall']):
    if entry == 1.0 or entry == 2.0:
        ratings.append(-1)
    elif entry == 3.0:
        ratings.append(0)
    elif entry == 4.0 or entry == 5.0:
        ratings.append(1)

In [4]:
""""
Split each review into sentences
and preprocess each sentence
"""
# stopwords = stopwords.words('english')

preprocessed_data = []
index = 0
for review in reviews_data['reviewText']:
    if(index % 1000 == 0):
        print(index)
    review_sentences = tokenizer.tokenize(review)
    for sentence in review_sentences:
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        sentence = sentence.lower()
        sentence = word_tokenize(sentence)
#         sentence = [word for word in sentence if not word in stopwords]
        sentence = ' '.join(sentence)
        if(sentence != ''):
            review = {}
            review["index"] = index
            review["sentence"] = sentence
            review["rating"] = ratings[index]
            preprocessed_data.append(review)
    index += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [5]:
print("Before preprocessing: ", reviews_data['reviewText'][233315])
print("------------------------------------------------")
print("After preprocessing: ", [d for d in preprocessed_data if d['index'] == 233315])
print(len(preprocessed_data))

Before preprocessing:  &#1055;&#1086; &#1090;&#1072;&#1082;&#1086;&#1081; &#1093;&#1086;&#1088;&#1086;&#1096;&#1077;&#1081; &#1094;&#1077;&#1085;&#1077; &#1087;&#1088;&#1077;&#1082;&#1088;&#1072;&#1089;&#1085;&#1099;&#1077; &#1095;&#1072;&#1089;&#1099; &#1087;&#1083;&#1102;&#1089; &#1089;&#1084;&#1077;&#1085;&#1085;&#1099;&#1077; &#1088;&#1077;&#1084;&#1077;&#1096;&#1082;&#1080; &#1084;&#1086;&#1103; &#1084;&#1072;&#1084;&#1072; &#1087;&#1088;&#1086;&#1089;&#1090;&#1086; &#1074; &#1074;&#1086;&#1089;&#1090;&#1086;&#1088;&#1075;&#1077;, &#1087;&#1088;&#1072;&#1074;&#1076;&#1072; &#1103; &#1080;&#1093; &#1077;&#1097;&#1077; &#1085;&#1077; &#1087;&#1086;&#1083;&#1091;&#1095;&#1080;&#1083; &#1085;&#1086; &#1091;&#1078;&#1077; &#1074; &#1087;&#1088;&#1077;&#1076;&#1074;&#1082;&#1091;&#1096;&#1077;&#1085;&#1080;&#1080; , &#1089;&#1082;&#1086;&#1088;&#1086; &#1076;&#1086;&#1081;&#1076;&#1091;&#1090; &#1080; &#1103; &#1085;&#1072;&#1087;&#1080;&#1096;&#1091; &#1077;&#1097;&#1077; &#1082;&#1072

In [6]:
# Split Dataset into training and test
indexes = [i for i in range(len(reviews_data))]

Train_X_index, Test_X_index, Train_Y_review, Test_Y_review = model_selection.train_test_split(indexes,ratings,test_size=0.3)

df = pd.DataFrame(preprocessed_data)

train = df[df['index'].isin(Train_X_index)]
Train_Y = train['rating'].tolist()
Train_X = train['sentence'].tolist()
Train_index = train['index'].tolist()


test = df[df['index'].isin(Test_X_index)]
Test_Y = test['rating'].tolist()
Test_X = test['sentence'].tolist()
Test_index = test['index'].tolist()

all_sentences = Train_X + Test_X

In [8]:
# Encoding of label
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
Train_Y_review = Encoder.fit_transform(Train_Y_review)
Test_Y_review = Encoder.fit_transform(Test_Y_review)

In [9]:
# Word Vectorization
        
Hashing_vect = HashingVectorizer(alternate_sign=False)
Hashing_vect.fit(all_sentences)
Train_X_Hashing = Hashing_vect.transform(Train_X)
Test_X_Hashing = Hashing_vect.transform(Test_X)

In [10]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Hashing,Train_Y)

# predict the labels on validation dataset
predictions_NB_sentences = Naive.predict(Test_X_Hashing)

# Get grades by review
predicted_scores_frame = pd.DataFrame(np.column_stack([Test_index, predictions_NB_sentences, Test_Y]), 
                               columns=['review Index', 'prediction', 'actual'])

print(len(predicted_scores_frame['review Index'].unique()) - len(Test_Y_review))

unique_indexes = predicted_scores_frame['review Index'].unique();

#find the majority score by review
reviews_sentiment = []
actual_sentiment = []
for item in unique_indexes:
    scores = list(predicted_scores_frame[predicted_scores_frame['review Index'] == item]['prediction'])
    majority_score = max(set(scores), key = scores.count)
    reviews_sentiment.append(majority_score)
    actual_sentiment.append(predicted_scores_frame[predicted_scores_frame['review Index'] == item]['actual'].unique()[0])
    
# Calculate Accuracy
accuracy = accuracy_score(reviews_sentiment, actual_sentiment)*100

# Confusion Matrix
cm = confusion_matrix(actual_sentiment, reviews_sentiment)

# Classification Report
my_tags = ['Negative','Neutral','Positive']
report = classification_report(actual_sentiment, reviews_sentiment, target_names=my_tags);




-11


C:\Users\user\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
cm

array([[   32,     0,  7970],
       [   10,     0,  9272],
       [    3,     0, 66306]], dtype=int64)